# Prelude

In [ ]:
# Enable interactive plots (%matplotlib -l to list backends)
%matplotlib inline

# print code version (hash of checked out version)
print("\nCurrent commit:")
!git log -1
print("\nChanges since last commit:")
!git status --short

# print date and time of script execution
import datetime
print("\nNotebook executed at at {} in following directory:".format(datetime.datetime.now()))
%cd ..

# Create cell

In [ ]:
import gunay_model
from neuron import h
import bluepyopt.ephys as ephys

cell = gunay_model.define_cell() # cell model description
nrnsim = ephys.simulators.NrnSimulator(dt=0.025, cvode_active=False)
cell.instantiate(sim=nrnsim)
icell = cell.icell # cell instantiated in Hoc

In [ ]:
named_seclists =  {listname: list(getattr(icell, listname)) for listname in cell.seclist_names}
for k, v in named_seclists.items():
    if len(v)==0:
        named_seclists.pop(k)
    else:
        print("{} : {} sections".format(k, len(v)))

In [ ]:
soma = named_seclists['somatic'][0]
axon = named_seclists['axonal'][0]

# Plot cell morphology

In [ ]:
from common import morphology
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

plt.figure()
ax = plt.subplot(111, projection='3d')
# ax.view_init(90,90) # set azimut and elevation
# ax.set_zlim((-1,1)) # prevent rounding errors
# ax.set_axis_off() # hide x,y,z axes and grid

# Plot shape of our cell
# h.define_shape(sec=soma) # for cells without 3D morphology
morphology.shapeplot(h, ax)

# Plot soma locations
morphology.mark_locations(h, soma, 0.5, label='soma', markspec='or')
morphology.mark_locations(h, axon, 1.0, label='axon_tip', markspec='og')


plt.legend()
plt.show()

# Experiment: spontaneous activity

In [ ]:
# start recording
vrec = h.Vector()
vrec.record(soma(0.5)._ref_v)
trec = h.Vector()
trec.record(h._ref_t)

# Init and run simulation
h.dt = 0.025
h.celsius = 35.0
h.v_init = -68.0
h.tstop = 500.0
h.init()
# h.run()
nrnsim.run(h.tstop, h.dt)

# plot results
plt.figure()
plt.plot(trec.as_numpy(), vrec.as_numpy())
plt.xlim(0, h.tstop)
plt.ylim(-80,60)
plt.xlabel('time (ms)')
plt.ylabel('$V_m$ (mV)')
plt.title('Spontaneous activity')

plt.show()